In [1]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [2]:
tokenized_corpus

[['Hello', 'there', 'good', 'man!'],
 ['It', 'is', 'quite', 'windy', 'in', 'London'],
 ['How', 'is', 'the', 'weather', 'today?']]

In [4]:
import pandas as pd
import nltk
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.corpus import wordnet
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn



## *******" gen and og"******** ##

def eval_gen_og(gen,og):
    score = 0
    for i in gen:
        best = 0
        w1 = wn.synsets(i)
        if (len(w1) > 0):
            for j in og:
                w2 = wn.synsets(j)
                if len(w2) < 1:
                    continue
                temp = w1[0].path_similarity(w2[0])
                if temp != None and temp > best:
                    best = temp
        else:
            if i in og:
                best = 1
        #     print(best)
        score = score + best

    return score / len(gen)



## ************"og and query "**************** ##
def eval_og_query(og,query):
    score = 0
    for i in og:
        best = 0
        w1 = wn.synsets(i)
        if(len(w1)>0):
            for j in query:
                w2 = wn.synsets(j)
                if len(w2)<1:
                    continue
                temp = w1[0].path_similarity(w2[0])
                if temp!= None and temp > best:
                    best = temp
        else:
            if i in query:
                best = 1
    #     print(best)
        score = score + best
    return score/len(og)

### ****** "gen and query " ***** ####
def eval_gen_query(gen,query):
    score = 0
    for i in gen:
        best = 0
        w1 = wn.synsets(i)
        if (len(w1) > 0):
            for j in query:
                w2 = wn.synsets(j)
                if len(w2) < 1:
                    continue
                temp = w1[0].path_similarity(w2[0])
                if temp != None and temp > best:
                    best = temp
        else:
            if i in query:
                best = 1
        #     print(best)
        score = score + best

    return score / len(gen)

if __name__ == "__main__":
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    stop = stopwords.words('english')
    df = pd.read_csv('input_for_eval.csv')
    print(df.head())
    df3_main = pd.read_csv('electronics1.csv', encoding='utf-8')
    ultimate_score  = []
    df3_main = df3_main[-200:]
    for i in df.index:
        asin = df.asin[i]
        # print("asin",asin)
        query = df['query'][i]
        # print("query",query)
        gen_questions = df.cosine[i]
        gen_questions2 = df.bm25[i]
        x = df3_main.loc[df3_main['asin'] == str(asin)]
        og_questions = x['question'].tolist()

        og = []
        gen = []
        gen2=[]
        
        for i in og_questions:
            text = tokenizer.tokenize(i)
            og.extend(text)
        for i in gen_questions:
            text = tokenizer.tokenize(i)
            gen.extend(text)
        for i in gen_questions2:
            text = tokenizer.tokenize(i)
            gen2.extend(text)
        og = [x.lower() for x in og]
        gen = [x.lower() for x in gen]
        gen2 = [x.lower() for x in gen2]
  
        og = list(set(og))
        gen = list(set(gen))
        gen2 = list(set(gen2))
        og = [word.strip('; : ! * - \\ · . ® \® //') for word in og if word not in (stop)]
        gen = [word.strip('; : ! * - \\ · . ® \® //') for word in gen if word not in (stop)]
        gen2 = [word.strip('; : ! * - \\ · . ® \® //') for word in gen2 if word not in (stop)]
        combined = gen.copy()
        combined.extend(gen2)
        ultimate_score.append([eval_gen_og(gen,og),eval_gen_og(gen2,og),eval_gen_query(gen,query),eval_gen_query(gen2,query),eval_og_query(og,query)])
#         ultimate_score_bm25.append([eval_gen_og(gen,og),eval_gen_query(gen,query),eval_og_query(og,query)])
    output_eval = pd.DataFrame(ultimate_score, columns=['comparison_cosine','comparison_bm25', 'model_cosine','model_bm25','original'])
    output_eval.to_csv('eval_output_cosine_bm25.csv')


   Unnamed: 0        asin                                              query  \
0           0  B00KMB7OK6  ['product', 'description', 'nset', 'music', 'f...   
1           1  B00KMRGF28  ['inspiron', '15', 'economical', 'member', 'fa...   
2           2  B00KMRGF3M  ['increase', 'storage', 'possibility', 'powerf...   
3           3  B00KNM763E  ['home', 'belkin', 'netcam', 'hd', 'deliver', ...   
4           4  B00KO6LA8Q  ['compact', 'video', 'audio', 'recorder', 'siz...   

                                              cosine  \
0  ['Metric system: Does anybody know if this dev...   
1  ['this keyboard work with OSX 10.9 Mavericks?'...   
2  ['Will it increase volume for senior who needs...   
3  ['Is it just for taking Videos?', 'wi fi only?...   
4  ['Is it just for taking Videos?', 'Does this c...   

                                                bm25  
0  ['Can i connect this to a DLink DIR-615 Wirele...  
1  ['Does this keyboard have keys that allow you ...  
2  ['will this be